# Example of KEO Taylor series expansion for methane

Here we use the following internal coordinates

1. $r_1$
2. $r_2$
3. $r_3$
4. $r_4$
5. $s_1 = (2\cos\alpha_{12} - \cos\alpha_{13} - \cos\alpha_{14} - \cos\alpha_{23} - \cos\alpha_{24} + 2\cos\alpha_{34})/\sqrt{12}$
6. $s_2=\cos\alpha_{13} - \cos\alpha_{14} - \cos\alpha_{23} + \cos\alpha_{24}) / 2$
7. $s_3=(\cos\alpha_{24} - \cos\alpha_{13}) / \sqrt{2}$
8. $s_4=(\cos\alpha_{23} - \cos\alpha_{14}) / \sqrt{2}$
9. $s_5=(\cos\alpha_{34} - \cos\alpha_{12}) / \sqrt{2}$,

where $r_i\equiv\text{C-H}_i$ and $\alpha_{ij}\equiv\text{H}_i\text{-C-H}_j$

In [79]:
import itertools

import jax
import jax.numpy as jnp
import numpy as np

from vibrojet.keo import Gmat, batch_Gmat, com, eckart
from vibrojet.taylor import deriv_list
from vibrojet.jet_prim import inv

jax.config.update("jax_enable_x64", True)

First, define a function to obtain five (six) $\cos\alpha_{ij}$ angular coordinates from the five symmetrized $s_k$ coordinates.

In [ ]:
def from_symmetrized_alpha_to_alpha(symm_cos_alpha, cos_alpha_ini, no_iter: int = 10):
    """Gradient-descend approach for obtaining six angular coordinates 'cos(alpha)'
    from their five symmetry-adapted combinations
    """

    def symmetrized_cos_alpha(cos_alpha):
        cos_alpha12, cos_alpha13, cos_alpha14, cos_alpha23, cos_alpha24 = cos_alpha

        # compute sixth angle cos_alpha34

        sin_alpha12 = jnp.sqrt(1 - cos_alpha12**2)
        sin_alpha13 = jnp.sqrt(1 - cos_alpha13**2)
        sin_alpha14 = jnp.sqrt(1 - cos_alpha14**2)

        cosbeta312 = (cos_alpha23 - cos_alpha12 * cos_alpha13) / (
            sin_alpha12 * sin_alpha13
        )

        cosbeta412 = (cos_alpha24 - cos_alpha12 * cos_alpha14) / (
            sin_alpha12 * sin_alpha14
        )

        cos_alpha34 = (
            cos_alpha13 * cos_alpha14
            + (
                cosbeta312 * cosbeta412
                - jnp.sqrt((1 - cosbeta312**2) * (1 - cosbeta412**2))
            )
            * sin_alpha13
            * sin_alpha14
        )

        # define five symmetrized combinations

        sym = jnp.array(
            [
                (
                    2 * cos_alpha12
                    - cos_alpha13
                    - cos_alpha14
                    - cos_alpha23
                    - cos_alpha24
                    + 2 * cos_alpha34
                )
                / jnp.sqrt(12),
                (cos_alpha13 - cos_alpha14 - cos_alpha23 + cos_alpha24) / 2,
                (cos_alpha24 - cos_alpha13) / jnp.sqrt(2),
                (cos_alpha23 - cos_alpha14) / jnp.sqrt(2),
                (cos_alpha34 - cos_alpha12) / jnp.sqrt(2),
            ]
        )
        return sym, cos_alpha34

    cos_alpha = cos_alpha_ini

    for _ in range(no_iter):
        sym, cos_alpha34 = symmetrized_cos_alpha(cos_alpha)
        jac = jax.jacrev(symmetrized_cos_alpha)(cos_alpha)[0]
        am = jac.T @ jac
        ai = inv(am)
        bm = (symm_cos_alpha - sym) @ jac
        cos_alpha = cos_alpha + ai @ bm

    stdev = jnp.sqrt(jnp.mean(jnp.square(symm_cos_alpha - sym)))
    return *cos_alpha, cos_alpha34, stdev

Define a function to compute the Cartesian coordinates of atoms from given internal coordinates ($r_1$,$r_2$,...,$s_5$).

In [84]:
# masses of C, H1, H2, H3, H4
masses = [12.0, 1.007825035, 1.007825035, 1.007825035, 1.007825035]

# Reference values of internal coordinates for defining the Eckart frame
r_ref = 1.0859638364
x0 = np.array([r_ref, r_ref, r_ref, r_ref, 0.0, 0.0, 0.0, 0.0, 0.0])


# Function that defines transformation from internal coordinates
# to Cartesian coordinates of atoms
@eckart(x0, masses)
@com(masses)
def internal_to_cartesian(internal_coords):
    r1, r2, r3, r4 = internal_coords[:4]  # C-H distances
    symm_cos_alpha = internal_coords[4:]  # symmetry-adapted combinations of cos(H-C-H)

    # determine five (six) H-C-H angles from symmetry-adapted combinations

    alpha_ini = jnp.array([109.5 * np.pi / 180] * 5)
    cos_alpha_ini = jnp.cos(alpha_ini)
    (
        cos_alpha12,
        cos_alpha13,
        cos_alpha14,
        cos_alpha23,
        cos_alpha24,
        cos_alpha34,
        stdev,
    ) = from_symmetrized_alpha_to_alpha(symm_cos_alpha, cos_alpha_ini)

    # define Cartesian coordinates of atoms

    sin_alpha12 = jnp.sqrt(1 - cos_alpha12**2)
    sin_alpha13 = jnp.sqrt(1 - cos_alpha13**2)
    sin_alpha14 = jnp.sqrt(1 - cos_alpha14**2)
    sin_alpha23 = jnp.sqrt(1 - cos_alpha23**2)
    sin_alpha24 = jnp.sqrt(1 - cos_alpha24**2)

    xyz_C = jnp.array([0.0, 0.0, 0.0])
    xyz_H1 = r1 * jnp.array([0.0, 0.0, 1.0])
    xyz_H2 = r2 * jnp.array([sin_alpha12, 0.0, cos_alpha12])

    v12 = xyz_H1
    v23 = xyz_H2
    n2 = v12 / jnp.linalg.norm(v12)
    n3 = jnp.cross(v23, v12)
    n3 = n3 / jnp.linalg.norm(n3)
    n1 = jnp.cross(n2, n3)

    cos_a = jnp.sum(n2 * v23) / jnp.linalg.norm(v23)
    sin_a = jnp.sqrt(1 - cos_a**2)
    cos_phi = (cos_alpha23 - cos_alpha13 * cos_a) / (sin_alpha13 * sin_a)
    sin_phi = jnp.sqrt(1 - cos_phi**2)

    xyz_H3 = r3 * (
        cos_alpha13 * n2 + sin_alpha13 * cos_phi * n1 + sin_alpha13 * sin_phi * n3
    )

    v12 = xyz_H1
    v23 = xyz_H2
    n2 = v12 / jnp.linalg.norm(v12)
    n3 = jnp.cross(v12, v23)
    n3 = n3 / jnp.linalg.norm(n3)
    n1 = jnp.cross(n3, n2)

    cos_a = jnp.sum(n2 * v23) / jnp.linalg.norm(v23)
    sin_a = jnp.sqrt(1 - cos_a**2)
    cos_phi = (cos_alpha24 - cos_alpha14 * cos_a) / (sin_alpha14 * sin_a)
    sin_phi = jnp.sqrt(1 - cos_phi**2)

    xyz_H4 = r4 * (
        cos_alpha14 * n2 + sin_alpha14 * cos_phi * n1 + sin_alpha14 * sin_phi * n3
    )

    return jnp.array([xyz_C, xyz_H1, xyz_H2, xyz_H3, xyz_H4])

# Print reference coordinates
xyz = internal_to_cartesian(x0)
print("Reference internal coordinates:\n", x0)
print("Reference Cartesian coordinates:\n", xyz)

Reference internal coordinates:
 [1.08596384 1.08596384 1.08596384 1.08596384 0.         0.
 0.         0.         0.        ]
Reference Cartesian coordinates:
 [[ 1.48053735e-18 -1.37356992e-19 -6.92534613e-18]
 [ 1.48053735e-18 -1.37356992e-19  1.08596384e+00]
 [ 1.02385652e+00 -1.37356992e-19 -3.61987945e-01]
 [-5.11928262e-01 -8.86685759e-01 -3.61987945e-01]
 [-5.11928262e-01  8.86685759e-01 -3.61987945e-01]]


In [ ]:
# Generate list of multi-indices specifying the integer exponents for each
# coordinate in the Taylor series expansion
max_order = 2  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(x0))])
    if sum(elem) <= max_order
]

# Function for computing kinetic G-matrix for given masses of atoms
# and internal coordinates
func = lambda x: Gmat(x, masses, internal_to_cartesian)

# Compute Taylor series expansion coefficients
Gmat_coefs = deriv_list(func, deriv_ind, x0, if_taylor=True)

print(Gmat_coefs.shape)  # (number of Taylor terms, ncoo+3+3, ncoo+3+3)

(55, 15, 15)


In [78]:
for i in range(len(deriv_ind)):
    print(deriv_ind[i], Gmat_coefs[i, 4,4], Gmat_coefs[i,5,5], Gmat_coefs[i,6,6], Gmat_coefs[i,7,7], Gmat_coefs[i,8,8])

(0, 0, 0, 0, 0, 0, 0, 0, 0) 75.64484824856237 75.64484824856238 61.72423540032705 61.7242354003271 61.72423540032705
(0, 0, 0, 0, 0, 0, 0, 0, 1) 4.1879848778908645e-14 3.630520672397978e-14 -4.062401455290519e-14 -3.2496751848534924e-14 -6.368810641893149e-13
(0, 0, 0, 0, 0, 0, 0, 0, 2) -23.88522847895151 -7.091704523302719 1.5512708557987427 1.5512708557986832 -33.16861155688618
(0, 0, 0, 0, 0, 0, 0, 1, 0) -6.629727144199975e-14 -2.333605071063071e-14 8.091481161718839e-14 -1.8481788898018274e-13 4.1701740555890575e-14
(0, 0, 0, 0, 0, 0, 0, 1, 1) -1.1723955140041653e-13 8.526512829121202e-14 6.543654507140673e-13 -1.8474111129762605e-13 -5.706546346573305e-14
(0, 0, 0, 0, 0, 0, 0, 2, 0) -11.290085512215132 -19.68684749003997 1.551270855798865 -33.16861155688511 1.5512708557986274
(0, 0, 0, 0, 0, 0, 1, 0, 0) -1.003778818458862e-13 -1.1357360140613382e-14 -1.7847516483387954e-13 8.28031293033587e-14 5.176029691921866e-14
(0, 0, 0, 0, 0, 0, 1, 0, 1) -1.6697754290362354e-13 6.394884621840